In [1]:
import pandas as pd

In [2]:
productDF = pd.read_csv('../data/lazada/product.csv')
reviewDF = pd.read_csv('../data/lazada/review.csv')

In [3]:
# extract anonymous review
nonAnonymousReviewDF = reviewDF[reviewDF['user_fk'] != 0]
nonAnonymousReviewDF.sample(5)

,comment,rating,sku,time,user,user_fk
34548,1. ส่งของรวดเร็วดีมาก ห่อแพ็คอย่างดี<br />\r\n...,5,UN355ELAA7LEZKANTH,2017-04-22 14:34:41,จารุพล ร.,430190
21909,สั่งวันที่ 22 23 ก็ได้เลยครับ ภาพสวย จอใหญ่ ปร...,5,HP466ELAA4BHKWANTH,2017-03-23 12:24:47,Manunchai Kruangjai,2096655
103823,NaN,3,DT888ELAA4P7Z2ANTH,2017-09-06 19:58:04,พัชรี น้ำพลอย,8180382
104445,สินค้าถือว่าดี มีคุณภาพ ในราคานี้คุ้มครับ,5,YU588ELAHBTJANTH,2017-07-02 20:55:50,ศิวะ โคจำนงค์,2080623
73843,ใช้ดีมากค่ะ ส่งสินค้าเร็วมาก แพจเกจสินค้า ดีงา...,1,CH843HBAA4374NANTH,2017-09-19 21:49:52,พรรณิการ์ ส.,5508933


In [4]:
# select only data complementing each other
productWithReviewDF = productDF[productDF['sku'].isin(nonAnonymousReviewDF['sku'])].copy()
reviewWithProductDF = nonAnonymousReviewDF[nonAnonymousReviewDF['sku'].isin(productDF['sku'])].copy()

In [5]:
print('shape of product df : {}'.format(productWithReviewDF.shape))
print('shape of review df : {}'.format(reviewWithProductDF.shape))

shape of product df : (7392, 4)
shape of review df : (76080, 6)


In [6]:
# drop duplicate
reviewWithProductDF = reviewWithProductDF[~reviewWithProductDF.duplicated(['sku', 'user_fk'])]
print('shape of review after drop dup : {}'.format(reviewWithProductDF.shape))

shape of review after drop dup : (75914, 6)


In [7]:
reviewWithProductCoreDF = reviewWithProductDF.drop(['comment', 'user'], axis=1)
reviewWithProductCoreDF.sample(5)

,rating,sku,time,user_fk
85421,5,ES654FAAA6M578ANTH,2017-09-28 19:31:35,7803553
102712,5,QU975HLAJAOXANTH,2016-01-25 16:04:03,1897391
119,4,CA182OTAA3JIRWANTH,2017-08-09 13:18:17,8418040
100439,4,EA994HAALWHIANTH,2015-08-17 16:34:24,89157
39753,1,NI919ELAA6JB4TANTH,2017-05-27 22:41:45,2646006


In [8]:
productWithReviewCoreDF = productWithReviewDF.drop(['url'], axis=1)
productWithReviewCoreDF.sample(5)

,name,price,sku
5341,Wacoal Panty Set 3 pcs. กางเกงในรูปทรง SHORT (...,299.0,WA811FAAA344K1ANTH
1721,SJCAM SJ4000 Action cam Car camera กล้องแอ็คชั...,1730.0,SJ883ELAA6E4O4ANTH
2873,กล้องจิ๋วขนาดเล็ก กล้องแอบถ่ายวีดีโอ กล้องสายล...,391.0,MI351ELAANKYGMANTH
12116,Mc Jeans Slim Fit Jeans MADY12000,1116.0,MC881FAAUDOYANTH
3332,ORICO กล่องเก็บ ฮาร์ดดิสก์ - รุ่น PHX-35 SET 5 สี,999.0,OR619ELACCR3ANTH


In [9]:
# select only product with review count > threshold
THRESHOLD = 3
productWithManyReviews = reviewWithProductCoreDF[['sku', 'user_fk']].groupby('sku').count()
productWithManyReviews = productWithManyReviews[productWithManyReviews['user_fk'] >= THRESHOLD].index.tolist()

# filter core DF
finalProductDF = productWithReviewCoreDF[productWithReviewCoreDF['sku'].isin(productWithManyReviews)]
finalReviewDF = reviewWithProductCoreDF[reviewWithProductCoreDF['sku'].isin(productWithManyReviews)]

In [10]:
# export
finalReviewDF.to_csv('../data/lazada/clean/review.csv', index=False, encoding='utf-8')
finalProductDF.to_csv('../data/lazada/clean/product.csv', index=False, encoding='utf-8')